# Outcome Prediction with RGAT Model

## 1. Load data from "processed_data" folder

In [19]:
import time
import joblib
import math
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import RGATConv
from torch_geometric.utils import to_undirected

In [20]:
num_patients = 1000
inverse_triples = True
embed_dim = 100
num_heads = 1

entity = pd.read_csv(f'processed_data/sphn_entities_{num_patients}_noOutcome.tsv', sep='\t', index_col=0, header=None)
entity = entity.to_dict()[1]

indices = []
for i in range(num_patients):
    idx = f'<http://nvasc.org/synth_patient_{i}>'
    indices.append(entity[idx])

events = pd.read_csv(f'processed_data/sphn_events_{num_patients}_noOutcome.tsv', sep='\t', header=None)
y = joblib.load(f'../Data Generation/outcomes_{num_patients}_0.joblib')

non_test_X, test_X, non_test_y, test_y_ = train_test_split(indices, y, stratify=y, test_size=0.2)
train_X, valid_X, train_y_, valid_y_ = train_test_split(non_test_X, non_test_y, stratify=non_test_y, test_size=1./8)

edge_index = torch.vstack((torch.Tensor(events[0]).long(),torch.Tensor(events[2]).long()))
edge_type = torch.Tensor(events[1]).long()
train_idx = torch.Tensor(train_X).long()
train_y = torch.Tensor(train_y_).long()
valid_idx = torch.Tensor(valid_X).long()
valid_y = torch.Tensor(valid_y_).long()
test_idx = torch.Tensor(test_X).long()
test_y = torch.Tensor(test_y_).long()
num_nodes = len(entity)

if inverse_triples == True:
    edge_index = to_undirected(edge_index)
    edge_type = torch.cat((edge_type, edge_type))

data = Data(
    edge_index=edge_index,
    edge_type=edge_type,
    train_idx=train_idx,
    train_y=train_y,
    valid_idx=valid_idx,
    valid_y=valid_y,
    test_idx=test_idx,
    test_y=test_y,
    num_nodes=num_nodes,
)
embedding = torch.nn.Parameter(torch.empty(num_nodes, embed_dim))
torch.nn.init.xavier_uniform_(embedding, gain=math.sqrt(2.0))
data.x = embedding
data.num_relations = data.num_edge_types
data.num_classes = 3
print(data)

Data(edge_index=[2, 221870], edge_type=[221870], train_idx=[700], train_y=[700], valid_idx=[100], valid_y=[100], test_idx=[200], test_y=[200], num_nodes=33563, x=[33563, 100], num_relations=10, num_classes=3)


## 2. Training the model

In [21]:
class RGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 num_relations):
        super().__init__()
        self.conv1 = RGATConv(in_channels, hidden_channels, num_relations, heads=num_heads)
        self.conv2 = RGATConv(hidden_channels*num_heads, out_channels, num_relations, heads=num_heads)
        # self.lin = torch.nn.Linear(hidden_channels*num_heads, out_channels)

    def forward(self, x, edge_index, edge_type):
        x = F.elu(self.conv1(x, edge_index, edge_type))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        # x = self.lin(x)
        return F.log_softmax(x, dim=-1)


if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = RGAT(embed_dim, 16, data.num_classes, data.num_relations).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [22]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    val_acc = float((pred[data.valid_idx] == data.valid_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, val_acc, test_acc

In [23]:
times = []
for epoch in range(1, 1001):
    start = time.time()
    loss = train()
    if epoch % 10 ==0:
        train_acc, val_acc, test_acc = test()
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f} | Train: {train_acc:.4f} '
            f'Val: {val_acc:.4f} Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 10, Loss: 1.0081 | Train: 0.4186 Val: 0.4300 Test: 0.4400
Epoch: 20, Loss: 0.9741 | Train: 0.4543 Val: 0.4900 Test: 0.4350
Epoch: 30, Loss: 0.9695 | Train: 0.5114 Val: 0.4200 Test: 0.4300
Epoch: 40, Loss: 0.9651 | Train: 0.4986 Val: 0.4300 Test: 0.4400
Epoch: 50, Loss: 0.9610 | Train: 0.5029 Val: 0.4500 Test: 0.4150
Epoch: 60, Loss: 0.9567 | Train: 0.5071 Val: 0.4400 Test: 0.4450
Epoch: 70, Loss: 0.9552 | Train: 0.5143 Val: 0.4700 Test: 0.4250
Epoch: 80, Loss: 0.9551 | Train: 0.5143 Val: 0.4500 Test: 0.4300
Epoch: 90, Loss: 0.9566 | Train: 0.5100 Val: 0.4500 Test: 0.4500
Epoch: 100, Loss: 0.9515 | Train: 0.5114 Val: 0.4600 Test: 0.4600
Epoch: 110, Loss: 0.9399 | Train: 0.5229 Val: 0.4400 Test: 0.4500
Epoch: 120, Loss: 0.9438 | Train: 0.5314 Val: 0.4400 Test: 0.4550
Epoch: 130, Loss: 0.9394 | Train: 0.5271 Val: 0.4400 Test: 0.4600
Epoch: 140, Loss: 0.9306 | Train: 0.5557 Val: 0.4500 Test: 0.4300
Epoch: 150, Loss: 0.9264 | Train: 0.5457 Val: 0.4500 Test: 0.4650
Epoch: 160, Loss: 0

In [24]:
from sklearn.metrics import roc_auc_score

model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.edge_type).cpu()
    prob = F.softmax(out, dim=1)
auc = roc_auc_score(data.test_y.cpu(), prob[data.test_idx.cpu()], multi_class='ovr')
print(f'ROC AUC score: {auc:.4f}')
# print(prob.shape)

ROC AUC score: 0.5303
